<a href="https://colab.research.google.com/github/hws2002/MachineLearning_PytorchNScikitLearn/blob/master/chapter4/chapter4_2_categorical_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [113]:
import pandas as pd
import numpy as np

# 4.2.1 판다스를 사용한 범주형 데이터 인코딩
범주형 데이터에 관해 이야기할 때 순서가 있는 것과 없는 것을 구분해야 함.  

In [114]:
df = pd.DataFrame([
    ['green', 'M', 10.1, 'class2'],
    ['red', 'L', 13.5, 'class1'],
    ['blue','XL',15.3, 'class2'],
])
df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class2
1,red,L,13.5,class1
2,blue,XL,15.3,class2


# 4.2.2 순서가 있는 특성 매핑
학습 알고리즘이 순서 특성을 올바르게 인식하려면 범주형의 문자열 값을 정수로 바꾸어야 함.  
하지만 size특성의 순서를 올바르게 자동으로 바꿔주는 함수는 없기 때문에, 매핑 함수를 직접 만들어야 함.  
만약 특성 간의 산술적인 차이를 이미 알고 있다 가정하면,  
예를 들어 XL = L + 1 = M + 2

In [115]:
size_mapping = {
    'XL': 3,
    'L': 2,
    'M': 1}

In [116]:
df['size'] = df['size'].map(size_mapping)
df['size']

0    1
1    2
2    3
Name: size, dtype: int64

In [117]:
# 만약 정수 값을 다시 원래 문자열 표현으로 바꾸고 싶다면 거꾸로 매핑하는 딕셔너리를 정의하면 됨.
inv_size_mapping = {v:k for k, v in size_mapping.items()}

In [118]:
df['size'].map(inv_size_mapping)

0     M
1     L
2    XL
Name: size, dtype: object

# 4.2.3 클래스 레이블 인코딩 (순서 x)
많은 머신 러닝 라이브러리는 클래스 레이블이 정수로 인코딩되었을 것이라고 기대함.  
사이킷런의 분류 추정기 대부분은 자체적으로 클래스 레이블을 정수로 변환해 주지만 사소한 실수를 방지하기 위해 클래스 레이블을 정수 배열로 전달하는 것이 좋은 습관임.  

**특정 문자열 레이블에 할당한 정수는 아무런 의미가 없음!**

In [119]:
class_mapping = { cls : idx for idx, cls in enumerate(np.unique(df['classlabel']))}
class_mapping
inv_class_mapping = {v : k for k , v in class_mapping.items()}

In [120]:
df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,1
1,red,2,13.5,0
2,blue,3,15.3,1


In [121]:
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
print(df)
# 다른 방법으로는 사이킷런에 구현된 LabelEncoder 클래스를 사용하면 편함
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
print(y)

   color  size  price classlabel
0  green     1   10.1     class2
1    red     2   13.5     class1
2   blue     3   15.3     class2
[1 0 1]


In [122]:
df['classlabel'] = y

In [123]:
print(df)
# inverse_transform을 이용해 정수 클래스 레이블을 원본 문자열 형태로 되돌릴 수 있음
class_le.inverse_transform(y)

   color  size  price  classlabel
0  green     1   10.1           1
1    red     2   13.5           0
2   blue     3   15.3           1


array(['class2', 'class1', 'class2'], dtype=object)

# 4.2.4 순서가 없는 특성에 원-핫 인코딩 적용
4.2.2절에서는 순서가 있는 특성들을 매핑했고,  
4.2.3절에서는 `LabelEncoder`를 사용하여 간편하게 문자열 레이블을 정수로 인코딩함  
순서가 없는 `color` 열에도 비슷한 방식을 적용해 보자.

In [124]:
X = df[['color', 'size', 'price']].values # df에서 color, size, price 열만 때오자
color_le = LabelEncoder()
X[:,0] = color_le.fit_transform(X[:,0])
X
# blue = 0, green = 1, red = 2로 매핑된것을 알 수 있음

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

### 여러개의 범주형 데이터열을 정수로 인코딩하기

* `OrdinalEncoder` : 범주형 데이터를 정수로 인코딩해줌
* `ColumnTransformer` : 판다스 데이터프레임의 열마다 다른 변환을 적용하도록 도와주ㅏㅁ

In [125]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder

ord_enc = OrdinalEncoder(dtype = int) # OrdinalEncoder 클래스의 dtype 매개변수 기본값은 np.float64로 실수를 인코딩함
col_trans = ColumnTransformer([('ord_enc', ord_enc, ['color'])])
X_trans = col_trans.fit_transform(df)
X_trans

array([[1],
       [2],
       [0]])

* `ColumnTransformer` 의 첫 매개변수는 transformer의 리스트임.
트랜스포머는 이름, 변환기, 변환할 열의 리스트로 이루어진 튜플.  
결과에서 알 수 있듯이 `color`열이 정수 값으로 변환되었고, 이를 변환시킨 transformer의 이름이 "ord_enc"라고 이해하면 됨.

In [126]:
# size열을 가지고도 비슷한 작업을 해보자!

# 우선 size열을 원래대로 되돌린다
df['size'] = df['size'].map(inv_size_mapping)

# OrdinalEncoder 변환기를 사용하여 size를 정수로 인코딩한다.
size_trans = ColumnTransformer([('size_enc', ord_enc, ['size'])])
X_s_trans = size_trans.fit_transform(df)
print(X_s_trans)
size_trans.named_transformers_['size_enc'].inverse_transform(X_s_trans)

# 기존의 매핑을 다시 적용한다.
df['size'].map(size_mapping)

[[1]
 [0]
 [2]]


0    1
1    2
2    3
Name: size, dtype: int64

In [127]:
col_trans.named_transformers_['ord_enc'].inverse_transform(X_trans)

array([['green'],
       ['red'],
       ['blue']], dtype=object)

In [128]:
# 같은 np.array에 대해 서로 다른 변환기를 사용하면 서로 다른 결과를 출력한다.
size_trans.named_transformers_['size_enc'].inverse_transform(X_trans)

array([['M'],
       ['XL'],
       ['L']], dtype=object)

## 원-핫 인코딩 (one-ho encoding)
순서 없는 특성에 들어 있는 고유한 값마다 새로운 더미(dummy) 특성을 만드는 것.  
사이킷런의 `preprocessing` 모듈에 구현된 `OneHotEncoder`를 사용하여 이런 변환을 수행할 수 있음

In [129]:
from sklearn.preprocessing import OneHotEncoder
X = df[['color', 'size', 'price']].values
color_ohe = OneHotEncoder()
color_ohe.fit_transform(X[:,0].reshape(-1,1)).toarray()
# 배열의 다른 두 열을 수정하지 않기 위해 OneHotEncoder를 하나의 열[:,0].reshape(-1,1)에만 적용함

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [130]:
# 여러 개의 특성이 있는 배열에서 특정 열만 반환하려면 ColumnTransformer를 사용하자
from sklearn.compose import ColumnTransformer
X = df[['color','size','price']].values
c_transf = ColumnTransformer([ # (name, transformer, column(s)) 튜플의 리스트를 받음
    ('onehot', OneHotEncoder(dtype = int), [0]), # 첫 번쨰 열만 변환하기 위해 지정했고,
    ('nothinh', 'passthrough', [1,2]) # 나머지 두 열은 변경하지 않고 그대로 두기 위해 'passthrough'로 지정함
])
c_transf.fit_transform(X)

array([[0, 1, 0, 'M', 10.1],
       [0, 0, 1, 'L', 13.5],
       [1, 0, 0, 'XL', 15.3]], dtype=object)

In [131]:
# 중복된 열을 삭제하려면 drop = 'first' 와 categories = 'auto'로 지정하자.
color_ohe = OneHotEncoder(categories = 'auto', drop = 'first')
c_transf = ColumnTransformer([
    ('onehot', color_ohe, [0]),
    ('nothing', 'passthrough', [1,2])
])

c_transf.fit_transform(X)

array([[1.0, 0.0, 'M', 10.1],
       [0.0, 1.0, 'L', 13.5],
       [0.0, 0.0, 'XL', 15.3]], dtype=object)

### 판다스의 `get_dummies` 메서드 이용해서 원-핫 인코딩하기
Dataframe에 적용하면 `get_dummies` 메서드는 문자열 열만 변환하고 나머지 열은 그대로 둠

In [132]:
print(df)
df['size'] = df['size'].map(size_mapping)
pd.get_dummies(df[['price','color','size']])

   color size  price  classlabel
0  green    M   10.1           1
1    red    L   13.5           0
2   blue   XL   15.3           1


,price,size,color_blue,color_green,color_red
0,10.1,1,False,True,False
1,13.5,2,False,False,True
2,15.3,3,True,False,False


In [133]:
# columns 매개변수를 사용하면 다음과 같이 변환하려는 특성을 구체적으로 지정할 수 있음
pd.get_dummies(df[['price','color','size']], columns=['size'])

,price,color,size_1,size_2,size_3
0,10.1,green,True,False,False
1,13.5,red,False,True,False
2,15.3,blue,False,False,True


* 원-핫 인코딩된 데이터셋을 사용하면 다중 공선성(multicollinearity) 문제가 생길 수 있음.  
이는 역행렬을 구해야하는 것과 같은 경우에 문제가 생길 수 있음.  
* 변수 간의 상관관계를 감소시키려면 원-핫 인코딩된 배열에서 특성 열 하나를 삭제시키자.  
이렇게 특성을 삭제해도 잃는 정보는 없음에 유의

In [ ]:
# get_dummies를 사용할 때 `drop_first`매개변수를 True로 지정하여 첫 번재 열을 삭제할 수 있음
pd.get_dummies(df[['price', 'size','color']],
               drop_first = True, columns = ['color', 'size']
               )

,price,color_green,color_red,size_2,size_3
0,10.1,True,False,False,False
1,13.5,False,True,True,False
2,15.3,False,False,False,True


### 순서가 있는 특서 인코딩하기
순서가 있는 특성의 범주 사이에서 수치적 크기에 대해 확신이 없거나 두 범주 사이의 순서를 정의할 수 없다면 임계 값을 사용하여 0/1로 인코딩할 수 있음.  
예를 들어 M, L, XL 값을 가진 특성 size를 두 개의 새로운 특성 'x > M' 과 'x > L'로 나눌 수 있음.

In [134]:
df = pd.DataFrame([
    ['green', 'M', 10.1, 'class2'],
    ['red', 'L', 13.5, 'class1'],
    ['blue', 'XL', 15.3, 'class2']
 ])
df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class2
1,red,L,13.5,class1
2,blue,XL,15.3,class2


In [135]:
df['x > M'] = df['size'].apply(lambda x : 1 if x in {'L', 'XL'} else 0)
df['x > L'] = df['size'].apply(lambda x : 1 if x == 'XL' else 0)
del df['size']
df

,color,price,classlabel,x > M,x > L
0,green,10.1,class2,0,0
1,red,13.5,class1,1,0
2,blue,15.3,class2,1,1
